In [1]:
import numpy as np
import multiprocessing as mp

In [2]:
from glob import glob
from simulations.concentrations import get_plugin_categorical_concentration, get_plugin_density_concentration

In [3]:
import re

def get_concentration_estimates(filename):
    
    sim_num = re.findall(r'.*\.([0-9]+)\.npz', filename)[0]
    
    npzfile = np.load(filename)
    droplets = npzfile['sample_sizes'].T
    
    number_droplets, number_strains = droplets.shape
    
    small_val_size = 10000
    small_val_iterations = number_droplets // small_val_size
    small_val_results = np.zeros((2, small_val_iterations))
    
    for iter_num in range(small_val_iterations):
        batch = droplets[iter_num*small_val_size:(iter_num+1)*small_val_size,:]
        small_val_results[0,iter_num] = get_plugin_density_concentration(batch)
        small_val_results[1,iter_num] = get_plugin_categorical_concentration(batch)
        
    med_val_size = 500000
    med_val_iterations = number_droplets // med_val_size
    med_val_results = np.zeros((2, med_val_iterations))
    
    for iter_num in range(med_val_iterations):
        batch = droplets[iter_num*med_val_size:(iter_num+1)*med_val_size,:]
        med_val_results[0,iter_num] = get_plugin_density_concentration(batch)
        med_val_results[1,iter_num] = get_plugin_categorical_concentration(batch)
        
    whole_sim_results = np.zeros(2)
    whole_sim_results[0] = get_plugin_density_concentration(droplets)
    whole_sim_results[1] = get_plugin_categorical_concentration(droplets)
    
    results = {"small_batches":small_val_results, "medium_batches":med_val_results, "whole_sim":whole_sim_results}
    np.savez_compressed('concentration_estimates/simulation.{}.npz'.format(sim_num), **results)

In [4]:
filenames = sorted(list(glob('npzfiles/*.npz')))

In [5]:
%%time
with mp.get_context("fork").Pool(50) as pool:
    pool.map(get_concentration_estimates, filenames, chunksize=1)
    pool.close()
    pool.join()

CPU times: user 1.31 s, sys: 2.58 s, total: 3.89 s
Wall time: 1h 13min 15s
